In [3]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Plot monthly data by the hours
def plotMonth (month,data_df):
    
    startIDX = [0,744,1416,2160,2880,3624,4344,5088,5832,6552,7296,8016]
    days = [31,28,31,30,31,30,31,31,30,31,30,31]
    name = ['January','February','March','April','May','June',
            'July','August','September','October','November','December']
    hourly = pd.DataFrame({'Ta' : np.zeros(24),'T' : np.zeros(24),'Turb' : np.zeros(24)})

    # Iterate through hourly data
    for i in range(startIDX[month-1],startIDX[month-1] + 24 * days[month-1]):
        hourly.loc[i%24,'Ta'] += EPW_df.loc[i,'DB']/days[month-1]
        hourly.loc[i%24,'T'] += EPW_df.loc[i,'T']/days[month-1]
        hourly.loc[i%24,'Turb'] += EPW_df.loc[i,'Turb']/days[month-1]
    
    plt.figure(month)
    plt.plot(hourly.index,hourly['Turb'],'k--')
    plt.plot(hourly.index,hourly['Ta'],'k')
    plt.ylabel('deg C')
    plt.xlabel('Hour of Day')
    plt.title(name[month-1] + ' Average')
    plt.legend(['Urban','Rural'])
    plt.grid()
    
    return None

# Generate urban temperature from rural temperature data using first-order lag
# https://en.wikipedia.org/wiki/Time_constant#Thermal_time_constant
# EPW assumes TMY2 format (need to update to include TMY3)

def getUrbanTemp (filename,a,e,depth,dens,Cp,k,veg,indoorT,bldDensity):

    # Reference: http://bigladdersoftware.com/epx/docs/8-3/auxiliary-programs/energyplus-weather-file-epw-data-dictionary.html
    EPW_df = pd.read_csv(filename,dtype='float',skiprows = 8,delimiter = ',', header=None,usecols = [1,2,3,4,6,7,12,13,21])
    EPW_df = EPW_df.rename(columns={1:"Month",2:"Day",3:"Hour",4:"Minute",6:"DB",7:"WB",12:"DWR",13:"GHR",21:"WindSpeed"})

    sigma = 5.67e-8
    T = EPW_df.loc[0,'DB']+273.15  # initial temp
    Tind = indoorT + 273.15        # indoor temp

    for index,row in EPW_df.iterrows():
        
        # Progress indicator
        if index % 10 == 0:
            clear_output(wait=True)
            print('%.1f' % (index / 8750 *100) , '% complete.')

        Ta = row['DB']+273.15            # Ambient rural temperature
        vWind = row['WindSpeed']         # Wind speed
        h = 5.8+3.7*vWind                       # Convective coefficient - from UWB (Bruno) (use either)
    #    h = 10.45 - vWind + 10 *vWind**0.5     # Convective coefficient - from engineering box reference

        Qint = k*(Tind-T)                                           # conductive heat transfer from indoor space
        Qsun = (a-veg[int(row['Month'])-1]) * row['GHR']            # solar 
        Qconv = hc*(Ta - T)                                         # convective
        Qinfra = e*(row['DWR'] - T**4*sigma)                        # infared 
        dT = (Qint + Qconv + Qsun + Qinfra)/(dens*Cp*depth)*3600    # total temperature change
        T += dT     # update temperature

        # Write new values to dataframe
        EPW_df.loc[index,'Qint'] = round(Qint,1)
        EPW_df.loc[index,'Qsun'] = round(Qsun,1)
        EPW_df.loc[index,'Qconv'] = round(Qconv,1)
        EPW_df.loc[index,'Qinfra'] = round(Qinfra,1)
        EPW_df.loc[index,'dT'] = round(dT,1)
        EPW_df.loc[index,'T'] = round(T-273.15,1)
        EPW_df.loc[index,'Turb'] = round((bldDensity*T+(1-bldDensity)*Ta)-273.15,1)  
            
    return EPW_df

In [2]:
# Case one - Boston
# Urban parameters
a = 0.75                # Average absorptivity of urban surfaces (0.6~0.8) 
e = 0.9                 # Average emissivity of urban surfaces (0.85~0.95)
depth = 0.5             # Urban surface characteristic Depth (0.5~1.0m)
dens = 2400             # kg/m^3 (https://en.wikipedia.org/wiki/Properties_of_concrete)
Cp = 960                # J/(kg-K) Concrete/Asphalt (http://www.engineeringtoolbox.com/specific-heat-solids-d_154.html)     
k = 1.5                 # Thermal mass conductivity from internal building volume (arbitrary)
indoorT = 20            # indoor temperature (deg C)
bldDensity = 0.5        # somewhat of an arbitrary number, with 1 = fully built, 0 = no built

# Effective vegetation coverage by month for evapotranspiration on urban surface
veg = [0,0,0,          # Jan,Feb,Mar
       0.1,0.1,0.2,    # Apr,May,Jun
       0.2,0.2,0.2,    # Jul,Aug,Sep
       0.1,0.0,0.0]    # Oct,Nov,Dec

filename = os.path.join('data','weather','EPW','USA_MA_Boston-Logan.725090_TMY2.epw')
EPW_df = getUrbanTemp(filename,a,e,depth,dens,Cp,k,veg,indoorT,bldDensity)

# Plot months (January, July)
plotMonth(1,EPW_df)
plotMonth(7,EPW_df)

IOError: File data\weather\EPW\USA_MA_Boston-Logan.725090_TMY2.epw does not exist

In [4]:
# Case two - Singapore
# Urban parameters
a = 0.6                 # Average absorptivity of urban surfaces (0.6~0.8) 
e = 0.9                 # Average emissivity of urban surfaces (0.85~0.95)
depth = 0.5             # Urban surface characteristic Depth (0.5~1.0m)
dens = 2400             # kg/m^3 (https://en.wikipedia.org/wiki/Properties_of_concrete)
Cp = 960                # J/(kg-K) Concrete/Asphalt (http://www.engineeringtoolbox.com/specific-heat-solids-d_154.html)     
k = 1                   # Thermal mass conductivity from internal building volume (arbitrary)
indoorT = 20            # indoor temperature (deg C)
bldDensity = 0.5        # somewhat of an arbitrary number, with 1 = fully built, 0 = no built

# Effective vegetation coverage by month for evapotranspiration on urban surface
veg = np.full(12,0.2)  # for tropical region with constant foliage cover

filename = os.path.join('data','weather','EPW','SGP_Singapore.486980_IWEC.epw')
EPW_df = getUrbanTemp(filename,a,e,depth,dens,Cp,k,veg,indoorT,bldDensity)

# Plot months (January, July)
plotMonth(1,EPW_df)
plotMonth(7,EPW_df)

IOError: File data\weather\EPW\SGP_Singapore.486980_IWEC.epw does not exist